Importing libraries

In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from Database import Database
from numpy import array
from sklearn.naive_bayes import MultinomialNB as NB
import sqlite3
import tqdm
import random

And the data.

If the currency would increases on the next period after the news is launched -> labeled + 1
If the currnecy would decreases on the next period after the news is launched -> labeled - 1

In [2]:
conn = sqlite3.connect('berita.db')
c = conn.cursor()
c.execute("SELECT Text,Sentiment,Date,Clock From Berita ")
result = c.fetchall()

In [3]:
X = []
y = []
for I in result:
    X.append(I[0])
    y.append(I[1])

A good evaluation metric should be directly motivated by the problem.

We find three charactherization:
    
    (1) Both false positive and false negative are harmful
    
    (2) The number of positive point and negative point fluctuates as condition of economy
    
    (3) The number of positive point and negative is not equal and the evaluation metric accomodates them.
    
We would like to think that mrc is the most suitable parameter compared to other parameters like F1 but also provides more intuitive information than AUROC.

In [4]:
def mrc(pred,Y):
    
    pred = array(pred)
    Y    = array(Y)
    
    TP, FP , TN, FN = 0,0,0,0
    
    for I in range(len(pred)):
        if pred[I] == Y[I]:
            if pred[I] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if pred[I] == -1:
                FN += 1
            else:
                FP += 1
    print(TP,FP,TN,FN)
    try:
        return ((TP*TN)-(FP*FN)) / ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(0.5)
    except:
        return 0

In [5]:
#Custom class for GradientBoosting
class Boosting():
    def __init__(self):
        self.clf = GB()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m].toarray()))
        return pred
#Custom class for K Nearest Neighbor
class Neighbors:
    def __init__(self):
        self.clf = KNC()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m]))
        return pred

In [6]:
clfOption = [Boosting(),LR(n_jobs = -1),NB(),LinearSVC(),Neighbors(),RFC()]

We empoy nested cross validation.

The first cross validation is to find the best feature selection

The second cross validation is to evaluate the model.

Both are 5-fold cross validation.

However, on the training set of each cross validation, we would under sample the majority class to ensure 50/50 balance of the training set.

In [7]:
def splitGroup(kFold):
    #Me too lazy to plug in sklearn
    assert len(X) == len(y)
    index = [int(I) for I in range(len(X))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [8]:
def balancedTrain(X,y):
    balancedX = []
    balancedY = []
    
    index = {}
    
    assert len(X) == len(y)
    
    for I in range(len(X)):
        if y[I] not in index:
            index[y[I]] = []
        index[y[I]].append(I)
    minimumPoint = min([len(I) for I in index.values()])    
    
    for I in index:
        chosen = random.sample(index[I],k = minimumPoint)
        for J in chosen:
            balancedX.append(X[J])
            balancedY.append(y[J])
    return balancedX, balancedY

In [9]:
#First cross validation
group = splitGroup(5)
XkFold = [[X[J] for J in K]for K in group]
YkFold = [[y[J] for J in K]for K in group]
counterList = []
selectList = []
mreTotal = []
t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J == I:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTest.append(newSentence)
            yTest.extend(YkFold[J])
        else:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTrain.append(newSentence)
            yTrain.extend(YkFold[J])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    counterList.append(CV(ngram_range=(2,2),min_df=5)) 
    trainVector = counterList[-1].fit_transform(xTrain)
    testVector  = counterList[-1].transform(xTest)
    
    selectList.append(SelectKBest(chi2, k = 10000))
    
    trainVector = selectList[-1].fit_transform(trainVector,yTrain)
    testVector  = selectList[-1].transform(testVector)
            
    
    mreTotal.append(0)
    for J in clfOption:
        J.fit(trainVector,yTrain)
        prediction = J.predict(testVector)
        mreTotal[-1] += mrc(prediction,yTest)
    t.update()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

6343 4567 2622 2486
5138 3083 4106 3691
5161 3202 3987 3668
5105 3153 4036 3724
5626 3906 3283 3203
4374 2586 4603 4455


 20%|████████████████▌                                                                  | 1/5 [05:26<21:46, 326.67s/it]

2910 1744 5411 5954
5154 2985 4170 3710
5181 3025 4130 3683
5190 3127 4028 3674
5220 3661 3494 3644
4529 2558 4597 4335


 40%|█████████████████████████████████▏                                                 | 2/5 [11:07<16:33, 331.04s/it]

...

 60%|████████████████████████████████████████████████                                | 3/5 [1:10:48<43:31, 1305.84s/it]

5964 4273 2957 2825
5185 3122 4108 3604
5178 3140 4090 3611
5188 3182 4048 3601
5125 3586 3644 3664
4484 2613 4617 4305


 80%|████████████████████████████████████████████████████████████████                | 4/5 [1:19:05<17:43, 1063.17s/it]

6026 4298 2953 2742
5170 3194 4057 3598
5036 3121 4130 3732
5178 3186 4065 3590
5291 3949 3302 3477
4315 2517 4734 4453


100%|█████████████████████████████████████████████████████████████████████████████████| 5/5 [1:26:16<00:00, 873.45s/it]

In [10]:
print("Average mre are")
print([I/5 for I in mreTotal])
print("The best feature selection is number %d"%(mreTotal.index(max(mreTotal))))

Average mre are
[0.14997606323208634, 0.1590669252163055, 0.14843724410587922, 0.15762721248819203, 0.14921693592123209]
The best feature selection is number 1


In [ ]:
index = mreTotal.index(max(mreTotal))

In [11]:
#Second cross validation to evaluate result
group = splitGroup(5)
XkFold = [[X[J] for J in K]for K in group]
YkFold = [[y[J] for J in K]for K in group]
mreTotal = [0 for I in clfOption]
t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J == I:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTest.append(newSentence)
            yTest.extend(YkFold[J])
        else:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTrain.append(newSentence)
            yTrain.extend(YkFold[J])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    trainVector = counterList[index].transform(xTrain)
    testVector  = counterList[index].transform(xTest)
        
    trainVector = selectList[index].transform(trainVector)
    testVector  = selectList[index].transform(testVector)

    for J in range(len(clfOption)):
        clfOption[J].fit(trainVector,yTrain)
        prediction = clfOption[J].predict(testVector)
        mreTotal[J] += mrc(prediction,yTest)
    t.update()


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


2325 1314 5940 6439
6168 2381 4873 2596
6044 2411 4843 2720
6126 2388 4866 2638
5330 3772 3482 3434
4477 2418 4836 4287


 20%|████████████████▌                                                                  | 1/5 [06:02<24:11, 362.99s/it]

6094 4361 2840 2724
6082 2332 4869 2736
6130 2418 4783 2688
6056 2391 4810 2762
5339 3694 3507 3479
4507 2399 4802 4311


 40%|█████████████████████████████████▏                                                 | 2/5 [12:26<18:27, 369.27s/it]

...

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [17:59<11:56, 358.36s/it]

6142 4407 2824 2646
6135 2398 4833 2653
6051 2468 4763 2737
6067 2425 4806 2721
5144 3609 3622 3644
4581 2346 4885 4207


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [23:23<05:47, 347.82s/it]

5906 4197 3088 2828
6116 2476 4809 2618
6044 2563 4722 2690
6040 2517 4768 2694
5075 3629 3656 3659
4550 2432 4853 4184


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [29:02<00:00, 345.33s/it]

In [12]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
assert len(name) == len(mreTotal)
for I in range(len(name)):
    print("%s -> %f" % (name[I],mreTotal[I]/5))

Gradient Boosting -> 0.097475
Logistic Regression -> 0.365742
Naive Bayes -> 0.350733
Linear SVC -> 0.354653
K nearest neighbor -> 0.087914
Random forest -> 0.184818


Chronological prediction

In [13]:
d = Database()
data = {}
label = {}
for I in result:
    session = d.cariSesi(I[2],I[3])[0]
    if session not in data:
        data[session] = []
        label[session] = I[1]
    data[session].append(I[0])


chronology = list(data.keys())

In [14]:
len(chronology)

492

In [15]:
def splitChronologyGroup(kFold):
    #Me too lazy to plug in sklearn
    index = [int(I) for I in range(len(chronology))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [16]:
#Second cross validation to evaluate result
group = splitChronologyGroup(5)
XkFold = [[chronology[J] for J in K]for K in group]
YkFold = [[label[chronology[J]] for J in K]for K in group]

In [19]:
mreTotal = [0 for I in clfOption]
#Second cross validation to evaluate result
group = splitChronologyGroup(5)
XkFold = [[chronology[J] for J in K]for K in group]
YkFold = [[label[chronology[J]] for J in K]for K in group]

t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J != I:
            for K in XkFold[J]:
                for L in data[K]:
                    sentence = L
                    newSentence = ""
                    for M in sentence:
                        if M.isalpha():
                            newSentence += M
                        else:
                            newSentence += " "
                    xTrain.append(newSentence)
                    yTrain.append(label[K])
        else:
            for K in XkFold[J]:
                xTest.append([])
                for L in data[K]:
                    sentence = L
                    newSentence = ""
                    for M in sentence:
                        if M.isalpha():
                            newSentence += M
                        else:
                            newSentence += " "
                    xTest[-1].append(newSentence)
                yTest.append(label[K])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    trainVector = counterList[index].transform(xTrain)
    testVector  = [counterList[index].transform(J) for J in xTest]
        
    trainVector = selectList[index].transform(trainVector)
    testVector  = [selectList[index].transform(J) for J in testVector]

    for J in range(len(clfOption)):
        clfOption[J].fit(trainVector,yTrain)
        prediction = [[-1,1][sum(clfOption[J].predict(K))>0] for K in testVector]
        mreTotal[J] += mrc(prediction,yTest)
    t.update()


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


42 39 6 11
52 4 41 1
48 8 37 5
52 5 40 1
47 33 12 6
16 7 38 37


 20%|████████████████▌                                                                  | 1/5 [05:51<23:24, 351.15s/it]

6 1 35 56
62 1 35 0
62 2 34 0
62 1 35 0
56 30 6 6
14 1 35 48


 40%|█████████████████████████████████▏                                                 | 2/5 [11:57<17:47, 355.69s/it]

...

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [17:41<11:44, 352.10s/it]

3 3 45 47
49 6 42 1
46 10 38 4
50 6 42 0
44 42 6 6
21 5 43 29


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [23:23<05:49, 349.26s/it]

2 2 46 49
50 0 48 1
48 0 48 3
51 1 47 0
47 40 8 4
20 3 45 31


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [29:57<00:00, 362.45s/it]

In [20]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
assert len(name) == len(mreTotal)
for I in range(len(name)):
    print("%s -> %f" % (name[I],mreTotal[I]/5))

Gradient Boosting -> 0.009783
Logistic Regression -> 0.919899
Naive Bayes -> 0.842482
Linear SVC -> 0.928164
K nearest neighbor -> 0.113635
Random forest -> 0.278932
